This code creates the matrices for the MPC model to be carried over to C++, it also performs LQR for comparison

In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy import sparse

import scipy.integrate as integrate
from matplotlib.patches import Rectangle
from numpy import sin, cos, pi, sign
import control
from scipy.integrate import solve_ivp

import scipy.integrate as integrate
import scipy.linalg as sc

%matplotlib notebook

In [2]:
#Model parameters
Lp = 0.129         #m
Mp = 0.024         #kg
Jp = 1/12*Mp*Lp**2
Bp = 0.0005

Lr = 0.085
Mr = 0.095 
g = 9.82
Jr = 1/12*Mr*Lr**2
Br = 0.0015

Rm = 8.4
kt = 0.042

dt = 0.05

In [3]:
#Constants to make the model matrices
T = (Mp*Lr**2+Jr)*(Jp + 1/4*Mp*Lp**2) + 1/4*(Mp*Lp*Lr)**2

a11 = Jp+1/4*Mp*(Lp**2)
a12 = -1/2*Mp*Lp*Lr
a21 = -a12
a22 = Jr + Mp*Lr**2

g0 = g/2*Mp*Lp

In [4]:
#Matrices
A = np.array([
        [0.0,       T,    0.0,     0.0],
        [0.0, -Br*a11, a12*g0, -a12*Bp],
        [0.0,     0.0,    0.0,       T],
        [0.0, -Br*a21, a22*g0,  -Bp*a22]
    ])
A = (1/T)*A

B = np.array([
        [0.0],
        [a11],
        [0.0],
        [a21]
    ])
B = (-kt/(T*Rm))*B   #Rescale to voltage

A_zoh = sc.expm(A*dt)
B_zoh = integrate.quad_vec(lambda x: sc.expm(A*x)@B, 0, dt)[0]

In [5]:
#LQR
Q = np.diag([10, 1, 10, 1])
R = np.diag([1]) 

K,S,_ = control.dlqr(A_zoh, B_zoh, Q, R)    #Run discrete LQR
Q_f = S                                     #Terminal cost from LQR

In [6]:
#The control gains
-K

array([[-1.39509209, -1.2058144 , 31.23476634,  2.73290132]])

In [7]:
N = 6

QN = sparse.block_diag([Q]*N + [Q_f]).toarray()  #Combining N Q's with 1 Q_f
RN = np.kron(np.eye(N), R)

In [8]:
F = np.vstack([np.linalg.matrix_power(A_zoh, i) for i in range(N+1)])
G = np.zeros((4*(N+1),N))

for m in range(N+1):
    for n in range(N):
        if m-1-n < 0:
            G[m,n] = 0
        else:
            G[4*m:4*m+4,n] = (np.linalg.matrix_power(A_zoh, m-n-1) @ B_zoh).T

In [9]:
#Constraints
H = G.T@QN@G + RN
f_theta = G.T@QN@F

Ax = np.zeros((6,4))
Ax[0,0], Ax[2,2] = (1,1)
Ax[1,0], Ax[3,2] = (-1,-1)
Ax = np.kron(np.eye(N+1), Ax)

Au = np.array([0,0,0,0,1,-1]).reshape(-1,1)
Au = np.kron(np.eye(N), Au)
Au = np.vstack((Au, np.zeros((6,N))))

#The first two values in b below determine the upper and lower bound for the arm's angle
#These values are set to both pi/2 and pi/6 in the thesis
b = np.array([pi/2, pi/2, pi/6, pi/6, 10, 10]).reshape(-1,1)  
b = np.vstack([b for i in range(N+1)])

A_constraints = Ax@G + Au
W = Ax@F

In [10]:
def MPC(init):
    u = cp.Variable((N, 1))
    epsilon = cp.Variable()
    
    #Define the cost function and constriants
    cost = 0.5*(cp.quad_form(u,H)) + ((f_theta@init).T)@u + epsilon**2
    constr = [(Ax@G + Au)@u <= (b-(Ax@F@init)) + epsilon]
    constr.append(epsilon >= 0)
    
    #Formulate and solve the optimization problem
    prob = cp.Problem(cp.Minimize(cost), constr)
    prob.solve(verbose=False)

    if prob.status == cp.OPTIMAL:
        ou = np.array((u.value[0, :])).flatten()[0]
    else:
        ou = None
    
    return ou         #Output u is only the first value out of the N computed u values

In [11]:
#Here we can try what the control input is for a given state 
#(should be similar to LQR when not close to a constriant)

state_init = np.array([1, 0, 0, 0])
MPC(state_init)

-1.3935526486801302